# Install Library

In [9]:
#!pip install pandas==1.0.4 #use pandas 1.0.4 to fix problem when install wml(require <=1.0.5)
!pip install ibm-cos-sdk
!pip install -U ibm-watson-machine-learning

     |████████████████████████████████| 10.1MB 3.8MB/s 
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.0.4 which is incompatible.
  Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5


     |████████████████████████████████| 61kB 3.3MB/s 
     |████████████████████████████████| 829kB 7.6MB/s 
     |████████████████████████████████| 143kB 28.6MB/s 
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.9.0-py2.py3-none-any.whl size=72656 sha256=d88940dc68f736957ac475d1521f14770f5438fd6114360b3bb1788a134081fb
  Stored in directory: /root/.cache/pip/wheels/a1/bd/d6/55a0a8486a324f3b96cd398ed2953892d46e63c99b36114f0c
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.9.0-py2.py3-none-any.whl size=498452 sha256=cbca89aa21635bc97280e67e9c6bc03f31074cd7051746f7161ef70b4cd67777
  Stored in directory: /root/.cache/pip/wheels/65/b6/8b/3d1586e529055db93bf453886e8215b52abbbc37169c1bc949
  Created wheel for ibm-cos-sdk-s3transfer: filename=ibm_cos_sdk_s3transfer-2.9.0-py2.py3-none-any.whl size=88604 sha256=38796072274ed153e724e5ad0423b7ecf925053150e5930c918c2c76234915c6
  Stored in directory: /root/.cache/pip/wheels/45/81/94/c6a58ec323b5dfbd9d1c380891be9d425cb39407

In [ ]:
!pip freeze

# Connect  WML

In [2]:
#WML Credentials
apikey = 'xCXcijMvlWzt0yS862yVKaBOy5boF9J8fc8g5TgZ9mmP'
location = 'us-south'

In [3]:
#Set WML authentication
wml_credentials = {
    "apikey": apikey,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

In [4]:
#Create client to connect to WML service via API, this connection is called 'client'
from ibm_watson_machine_learning import APIClient
client = APIClient(wml_credentials)

# Connect  COS

In [5]:
#Create cos credentials
COS_CRN = "crn:v1:bluemix:public:cloud-object-storage:global:a/a2fbcfc73145484da67dd67dbc4f914d:c9ef8a5f-f92d-4fd7-a5a8-8cb94bb46b28::"
WML_INST_NAME = "Machine Learning-1x"
WML_CRN = "crn:v1:bluemix:public:pm-20:us-south:a/a2fbcfc73145484da67dd67dbc4f914d:ab896b3d-9905-495f-b828-2593b75195c5::"

# Space

In [ ]:
client.spaces.ConfigurationMetaNames.show()

--------------  ----  --------
META_PROP NAME  TYPE  REQUIRED
NAME            str   Y
DESCRIPTION     str   N
STORAGE         dict  N
COMPUTE         dict  N
--------------  ----  --------


In [ ]:
#set space metadata
space_meta = {
    client.spaces.ConfigurationMetaNames.NAME: "Test to set3",
    ''' #when create new space this code will be generate new bucket in COS
    client.spaces.ConfigurationMetaNames.STORAGE: {
         "type": "bmcos_object_storage",
         "resource_crn": COS_CRN
     },
    '''
    client.spaces.ConfigurationMetaNames.COMPUTE: {
        'name': WML_INST_NAME,
        'crn': WML_CRN
    }
}

#create new space
space = client.spaces.store(space_meta)
space

Space has been created. However some background setup activities might still be on-going. Check for 'status' field in the response. It has to show 'active' before space can be used. If its not 'active', you can monitor the state with a call to spaces.get_details(space_id)


{'entity': {'compute': [{'crn': 'crn:v1:bluemix:public:pm-20:us-south:a/a2fbcfc73145484da67dd67dbc4f914d:ab896b3d-9905-495f-b828-2593b75195c5::',
    'guid': 'ab896b3d-9905-495f-b828-2593b75195c5',
    'name': 'Machine Learning-1x',
    'type': 'machine_learning'}],
  'members': [{'id': 'IBMid-550009U7HH',
    'role': 'admin',
    'state': 'active',
    'type': 'user'}],
  'name': 'Test to set3',
  'scope': {'bss_account_id': 'a2fbcfc73145484da67dd67dbc4f914d'},
  'status': {'state': 'preparing'}},
 'metadata': {'created_at': '2021-01-09T15:46:34.284Z',
  'creator_id': 'IBMid-550009U7HH',
  'id': '6bd5b37e-260e-4824-88cd-140d4aedbcb4',
  'url': '/v2/spaces/6bd5b37e-260e-4824-88cd-140d4aedbcb4'}}

In [7]:
#define space id
#space_id = client.spaces.get_id(space)
space_id = "35e5769e-3a6a-4eba-8fa0-09ebd594f205"
space_id

'35e5769e-3a6a-4eba-8fa0-09ebd594f205'

In [ ]:
client.spaces.get_details(space_id)

{'entity': {'compute': [{'crn': 'crn:v1:bluemix:public:pm-20:us-south:a/a2fbcfc73145484da67dd67dbc4f914d:ab896b3d-9905-495f-b828-2593b75195c5::',
    'guid': 'ab896b3d-9905-495f-b828-2593b75195c5',
    'name': 'Machine Learning-1x',
    'type': 'machine_learning'}],
  'name': 'Test Linear Regression',
  'scope': {'bss_account_id': 'a2fbcfc73145484da67dd67dbc4f914d'},
  'status': {'state': 'active'},
  'storage': {'properties': {'bucket_name': '56e34fe7-4cea-4f07-abb2-5b04b8838326',
    'bucket_region': 'us-south',
    'credentials': {'admin': {'access_key_id': '26d06c94cd0245edb499c4483ccc9864',
      'api_key': 'qjui5NpYAtxs0uU3R7AnJtp4I9a__Am2pbJXE4ZhkJR_',
      'secret_access_key': 'f6bb5a12c84c659d3322a43a233d173476c3c4f8bf279013',
      'service_id': 'ServiceId-fe413cf6-a24d-48dc-9e15-b174a32a92ac'},
     'editor': {'access_key_id': '29a1fcc9003b420bbe44c4c735a70827',
      'api_key': 'bZh6XJCBHg1OiKtkNwINtC19G4QXdH2W1F6QBsS6TEHf',
      'resource_key_crn': 'crn:v1:bluemix:public

In [9]:
#set default space
space_id = '35e5769e-3a6a-4eba-8fa0-09ebd594f205'
client.set.default_space(space_id)

'SUCCESS'

In [ ]:
#get space list
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ----------------------  ------------------------
ID                                    NAME                    CREATED
9fc43a8e-6979-4937-b26c-114d2fb9280c  Test                    2021-01-09T15:24:53.123Z
35e5769e-3a6a-4eba-8fa0-09ebd594f205  Test Linear Regression  2021-01-04T16:46:17.894Z
2873be61-8421-476c-9410-a48f46cd8605  test upload to wml      2021-01-04T02:28:36.976Z
1dcd2034-0d04-4e0e-83a1-05adf0d064c9  salegas5-deployments    2020-12-06T13:30:33.903Z
------------------------------------  ----------------------  ------------------------


# Model Definition

In [ ]:
client.model_definitions.ConfigurationMetaNames.show()

--------------  ----  --------  ----------------------------------------------------------------
META_PROP NAME  TYPE  REQUIRED  SCHEMA
NAME            str   Y
DESCRIPTION     str   N
PLATFORM        dict  Y         {'name(required)': 'string', 'versions(required)': ['versions']}
VERSION         str   Y
COMMAND         str   N
CUSTOM          dict  N
SPACE_UID       str   N
--------------  ----  --------  ----------------------------------------------------------------


In [9]:
#set model definition metadata
modeldef_meta = {
    client.model_definitions.ConfigurationMetaNames.NAME: "LR xgboost",
    client.model_definitions.ConfigurationMetaNames.PLATFORM: {'name': 'python', 'versions': ['3.7.7']},
    client.model_definitions.ConfigurationMetaNames.VERSION: "1.0"
}
#store model.zip file from local to WML
modeldef = client.model_definitions.store(r"D:\Alpha Com\New Platform\Model\linear regression_xgboost.zip", modeldef_meta)
modeldef

{'metadata': {'space_id': '35e5769e-3a6a-4eba-8fa0-09ebd594f205',
  'guid': '08eed2de-8378-4ff8-9355-7f88265224e2',
  'asset_type': 'wml_model_definition',
  'created_at': '2021-01-10T13:05:26Z',
  'last_updated_at': '2021-01-10T13:05:26Z',
  'name': 'LR xgboost',
  'href': '/v2/assets/wml_model_definition/08eed2de-8378-4ff8-9355-7f88265224e2?space_id=35e5769e-3a6a-4eba-8fa0-09ebd594f205'},
 'entity': {'wml_model_definition': {'version': '1.0',
   'platform': {'name': 'python', 'versions': ['3.7.7']}}}}

In [10]:
#define model definition id
modeldef_id = client.model_definitions.get_id(modeldef)
modeldef_id

'08eed2de-8378-4ff8-9355-7f88265224e2'

In [ ]:
#detail of model definition
client.model_definitions.get_details(modeldef_id)

{'entity': {'wml_model_definition': {'platform': {'name': 'python',
    'versions': ['3.7.7']},
   'version': '1.0'}},
 'metadata': {'asset_type': 'wml_model_definition',
  'attachment_id': '11ee156f-b630-4e7b-9405-3e8126730f96',
  'created_at': '2021-01-09T17:39:32Z',
  'guid': '5f0bbbfc-5f1c-43a7-b40c-936fbf8b8670',
  'href': '/v2/assets/wml_model_definition/5f0bbbfc-5f1c-43a7-b40c-936fbf8b8670?space_id=35e5769e-3a6a-4eba-8fa0-09ebd594f205',
  'last_updated_at': '2021-01-09T17:39:37Z',
  'name': 'LR xgboost (test3)',
  'space_id': '35e5769e-3a6a-4eba-8fa0-09ebd594f205'}}

In [ ]:
#get model definition list
client.model_definitions.list()

---------------------  --------------------  ------------------------------------
NAME                   ASSET_TYPE            GUID
LR xgboost (test)      wml_model_definition  884b9799-daf2-4c36-be10-4137ba714ac1
LR xgboost (test3)     wml_model_definition  609df4cd-bec0-4b83-8e62-b23e074468fb
XGB_tul with GPU       wml_model_definition  6150bbc6-c5ba-44be-b616-0962f14e6453
LR xgboost (test3)     wml_model_definition  5f0bbbfc-5f1c-43a7-b40c-936fbf8b8670
Generator 5 Model      wml_model_definition  1e491661-6ea0-4586-a48f-5ef28069b312
XGB_tul no GPU         wml_model_definition  32ecb769-b6ca-43f4-8b7e-0bc230c70b4a
LR xgboost (test)      wml_model_definition  bafc9f44-d496-45b3-81a2-c911655b6c57
LR xgboost (test)      wml_model_definition  21a14735-12df-482e-aad0-2df37989c716
LR xgboost             wml_model_definition  7f7e6507-414f-48d1-8145-eb9de7e219d6
LR xgboost-save model  wml_model_definition  65763f64-e4a9-4677-988e-90f86b341e81
---------------------  --------------------  ---

# Training Model Definition

In [150]:
client.training.ConfigurationMetaNames.show()

--------------------------  ----  --------  ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
META_PROP NAME              TYPE  REQUIRED  SCHEMA
TRAINING_DATA_REFERENCES    list  Y         [{'name(optional)': 'string', 'type(required)': 'string', 'connection(required)': {'endpoint_url(required)': 'string', 'access_key_id(required)': 'string', 'secret_access_key(required)': 'string'}, 'location(required)': {'bucket': 'string', 'path': 'string'}, 'schema(optional)': {'id(required)': 'string', 'fields(required)': [{'name(required)': 'string', 'type(required)': 'string', 'nullable(optional)': 'string'}]}}]
TRAINING_RESULTS_RE

In [ ]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029a-cc97-56da-b8e0-39c3880dbbe7  base
tensorflow_2.1-py3.6           1eb25b8

In [ ]:
client.hardware_specifications.list()

-------------  ------------------------------------  --------------------------------------------------------------------------------------------------------------------------------------
NAME           ID                                    DESCRIPTION
ML             5a1f0e64-e420-55ce-bd7a-f6d00bb942cf  A hardware specification providing 4 CPU cores and 32 GiB of memory.
V100x2         a02f3ab5-6964-4f06-a870-c7cc69187895  A hardware specification providing 52 CPU cores and 96 GiB of memory with 2 Nvidia v100 GPUs.
L              a6c4923b-b8e4-444c-9f43-8a7ec3020110  A hardware specification providing 8 CPU cores and 32 GiB of memory.
Default Spark  ac59d20a-9c7c-4504-a853-788ef35969da  A hardware specification for Spark with 1 CPU and 4 GiB of memory for master and worker nodes, with 2 workers.
XXS            b128f957-581d-46d0-95b6-8af5cd5be580  A hardware specification providing one CPU core and 2 GiB of memory.
M-Spark        b2232f7a-bfad-4822-9bce-6ba1af49217a  A hardware specif

In [11]:
#create training model definition (feed input data and get model metrics, model file)
training_meta = {
client.training.ConfigurationMetaNames.DESCRIPTION: "Value prediction",
client.training.ConfigurationMetaNames.NAME: "Sale Gas 5 Model",

client.training.ConfigurationMetaNames.TRAINING_RESULTS_REFERENCE: {
    "name": "result",
    "connection": {
          'endpoint_url': 'https://s3.us-south.cloud-object-storage.appdomain.cloud',
          'access_key_id': 'd9089fbf61194020b3601d7a24c5663e',
          'secret_access_key': '04d89ef816e54bff698cc2c85b1caa61434314fe5a8eb4a5'
      },
      "location": {
        "bucket": "training-model-result",
    },
    "type": "s3"
  },

 client.training.ConfigurationMetaNames.MODEL_DEFINITION: {
    "command": "pip install xgboost==0.90 && python 'linear regression_xgboost.py'",
    #"name": model_def_detail["metadata"]["name"],
    "id": modeldef_id,
    "hardware_spec": {
      "name": "k80",
      "nodes": 1
    },
    "software_spec": {
      "name": "pytorch-onnx_1.3-py3.7"
    },
     "parameters": {
          "name": "LR xgboost Model",
          "description": "A linear regression model using xgboost",
          #"args": arg
        }
  },
    client.training.ConfigurationMetaNames.TRAINING_DATA_REFERENCES: [
       {
      "name": "data",
      "type": "s3",
      "connection": {
          'endpoint_url': 'https://s3.us-south.cloud-object-storage.appdomain.cloud',
          'access_key_id': 'd9089fbf61194020b3601d7a24c5663e',
          'secret_access_key': '04d89ef816e54bff698cc2c85b1caa61434314fe5a8eb4a5'
      },
      "location": {
        "bucket": "training-model-data"
      }
    }
    ]
}
training = client.training.run(training_meta)
training

{'metadata': {'created_at': '2021-01-10T13:05:51.541Z',
  'description': 'Value prediction',
  'guid': 'a63c3040-c430-4b97-8325-14260af3a1c7',
  'id': 'a63c3040-c430-4b97-8325-14260af3a1c7',
  'name': 'Sale Gas 5 Model',
  'space_id': '35e5769e-3a6a-4eba-8fa0-09ebd594f205'},
 'entity': {'description': 'Value prediction',
  'model_definition': {'command': "pip install xgboost==0.90 && python 'linear regression_xgboost.py'",
   'hardware_spec': {'name': 'k80'},
   'id': '08eed2de-8378-4ff8-9355-7f88265224e2',
   'parameters': {'description': 'A linear regression model using xgboost',
    'name': 'LR xgboost Model'},
   'software_spec': {'name': 'pytorch-onnx_1.3-py3.7'}},
  'name': 'Sale Gas 5 Model',
  'results_reference': {'location': {'path': '',
    'training': '/a63c3040-c430-4b97-8325-14260af3a1c7',
    'training_status': '/a63c3040-c430-4b97-8325-14260af3a1c7/training-status.json',
    'bucket': 'training-model-result',
    'assets_path': '/a63c3040-c430-4b97-8325-14260af3a1c7/ass

In [13]:
#get training id
training_id = client.training.get_id(training)
training_id

'a63c3040-c430-4b97-8325-14260af3a1c7'

In [74]:
#get trining details
client.training.get_details(training_id)

{'entity': {'description': 'Value prediction',
  'model_definition': {'command': "pip install sklearn xgboost && python 'linear regression_xgboost.py'",
   'hardware_spec': {'name': 'k80'},
   'id': '611fc188-27bb-4430-8028-88e29d7755c0',
   'parameters': {'description': 'A linear regression model using xgboost',
    'name': 'LR xgboost Model'},
   'software_spec': {'name': 'pytorch-onnx_1.3-py3.7'}},
  'name': 'Sale Gas 5 Model',
  'results_reference': {'connection': {'access_key_id': 'd9089fbf61194020b3601d7a24c5663e',
    'endpoint_url': 'https://s3.us-south.cloud-object-storage.appdomain.cloud',
    'secret_access_key': '04d89ef816e54bff698cc2c85b1caa61434314fe5a8eb4a5'},
   'location': {'assets_path': '/d4a38991-3c18-43e4-9657-b28b602f82dc/assets',
    'bucket': 'training-model-result',
    'path': '',
    'training': '/d4a38991-3c18-43e4-9657-b28b602f82dc',
    'training_status': '/d4a38991-3c18-43e4-9657-b28b602f82dc/training-status.json'},
   'type': 's3'},
  'space': {'href': 

In [ ]:
client.training.list()

------------------------------------  ------  ------------------------
GUID (training)                       STATE   CREATED
1d24d2aa-ce91-4311-b53a-4aa5746c93b6  failed  2021-01-09T10:46:05.941Z
0e302fae-002f-48db-b990-eb08dc09c77e  failed  2021-01-09T04:25:02.677Z
17007873-3441-4780-a38f-c182ad18b54e  failed  2021-01-08T04:18:46.655Z
171111ab-781b-4afe-88da-c9485259425d  failed  2021-01-08T04:10:07.383Z
0a110355-1aca-4289-98bd-477e41848e7d  failed  2021-01-07T07:05:23.103Z
0838478b-e3f2-4da7-b9b5-d6923a52c9b5  failed  2021-01-07T07:02:40.489Z
17e35cfa-7b74-46f6-acf1-4b36185faf13  failed  2021-01-05T10:30:57.307Z
09cc1c63-7326-4be4-84ba-47de801cc624  failed  2021-01-05T10:06:24.554Z
198a83fa-3c5d-4eea-aa2b-d730cd96e4a5  failed  2021-01-05T09:36:26.442Z
14ad6b57-7b14-48e3-bd24-79534c01ba3c  failed  2021-01-05T08:49:23.338Z
------------------------------------  ------  ------------------------


In [14]:
#get training log
client.training.monitor_logs(training_id)



##########################################################################

Log monitor started for training run: a63c3040-c430-4b97-8325-14260af3a1c7

##########################################################################


{'text': 'DL execution ID is training-Ln2Uq8aMR', 'level': 'info'}
{'text': 'Training with training/test data at:\n', 'level': 'info'}
{'text': '  DATA_DIR: /mnt/data/training-model-data\n', 'level': 'info'}
{'text': '  MODEL_DIR: /job/model-code\n', 'level': 'info'}
{'text': "  TRAINING_COMMAND: pip install xgboost==0.90 && python 'linear regression_xgboost.py'\n", 'level': 'info'}
{'text': '  LOG_DIR: /job/logs\n', 'level': 'info'}
{'text': '  RESULT_DIR: /mnt/results/training-model-result/training-Ln2Uq8aMR\n', 'level': 'info'}
{'text': '  NUM_LEARNERS: 1\n', 'level': 'info'}
{'text': 'Sun Jan 10 13:07:04 UTC 2021: Running PyTorch job\n', 'level': 'info'}
{'text': 'Defaulting to user installation because normal site-packages is not writeable\n', 'level': '

In [15]:
#get saved model path 
model_path = client.training.get_details(training_id)['entity']['results_reference']['location']['logs']
model_path

'training-Ln2Uq8aMR'

In [ ]:
#for cance training
client.training.cancel(training_id)

####**-------------------------------------------------------------------**

# Download Trained Model & Store to WML

In [9]:
#connect to Cloud Object Storage
import ibm_boto3
from ibm_botocore.client import Config, ClientError

# Constants for IBM COS values
COS_ENDPOINT = "https://s3.us-south.cloud-object-storage.appdomain.cloud" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints
COS_API_KEY_ID = "w9OYjZ0qricse-Z0Sar08ccWq6XQw_3HmG4uUzA6qXwm" # eg "W00YixxxxxxxxxxMB-odB-2ySfTrFBIQQWanc--P3byk"
COS_INSTANCE_CRN = "crn:v1:bluemix:public:cloud-object-storage:global:a/a2fbcfc73145484da67dd67dbc4f914d:c9ef8a5f-f92d-4fd7-a5a8-8cb94bb46b28::" # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003xxxxxxxxxx1c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"

# Create client 
cos = ibm_boto3.client("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_INSTANCE_CRN,
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

In [11]:
bucket_name = "training-model-result" 
key = "training-seAzrEaGg/trained_model.h5"
res=cos.download_file(Bucket=bucket_name,Key=key,Filename='trained_model.h5')

In [26]:
#download model to notebook via cos
res=cos.download_file(Bucket=bucket_name,Key=key,Filename='/trained_model.h5')
trained_model = cos.get_object(Bucket=bucket_name, Key=model_path+'/trained_model.h5')

In [27]:
import xgboost as xgb

In [31]:
#load mode to xgb
model = xgb.XGBRegressor()
model.load_model('trained_model.h5')

# Store Model to WML

In [42]:
client.repository.ModelMetaNames.show()

------------------------  ----  --------  ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
META_PROP NAME            TYPE  REQUIRED  SCHEMA
NAME                      str   Y
DESCRIPTION               str   N
INPUT_DATA_SCHEMA         list  N         {'id(required)': 'string', 'fields(required)': [{'name(required)': 'string', 'type(required)': 'string', 'nullable(optional)': 'string'}]}
TRAINING_DATA_REFERENCES  list  N         [{'name(optional)': 'string', 'type(required)': 'string', 'connection(required)': {'endpoint_url(required)': 'string', 'access_key_id(required)': 'string', 'secret_access_key(required)': 'string'},

In [33]:
#store model to deployment asset
#https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_latest/wsj/wmls/wmls-deploy-python-types.html
sofware_spec_uid = client.software_specifications.get_id_by_name("default_py3.7")
model_meta = {
            client.repository.ModelMetaNames.NAME: 'LR xgboost Model',
            client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

model = client.repository.store_model(model=model ,meta_props=metadata)
model

{'entity': {'software_spec': {'id': 'e4429883-c883-42b6-87a8-f419d64088cd',
   'name': 'default_py3.7'},
  'type': 'scikit-learn_0.23'},
 'metadata': {'created_at': '2021-01-10T13:19:18.480Z',
  'id': '3eb9e4f2-a002-4367-b3c7-b41bb2d04fc8',
  'modified_at': '2021-01-10T13:19:22.543Z',
  'name': 'LR xgboost Model',
  'owner': 'IBMid-550009U7HH',
  'space_id': '35e5769e-3a6a-4eba-8fa0-09ebd594f205'},
 'system': {'warnings': []}}

In [48]:
#get model uid
model_uid = client.repository.get_model_uid(model)
model_uid

'3eb9e4f2-a002-4367-b3c7-b41bb2d04fc8'

In [66]:
#get model detail
client.repository.get_details(model_uid)

{'entity': {'software_spec': {'id': 'e4429883-c883-42b6-87a8-f419d64088cd',
   'name': 'default_py3.7'},
  'type': 'scikit-learn_0.23'},
 'metadata': {'created_at': '2021-01-10T13:19:18.480Z',
  'id': '3eb9e4f2-a002-4367-b3c7-b41bb2d04fc8',
  'modified_at': '2021-01-10T13:19:22.543Z',
  'name': 'LR xgboost Model',
  'owner': 'IBMid-550009U7HH',
  'space_id': '35e5769e-3a6a-4eba-8fa0-09ebd594f205'},
 'system': {'warnings': []}}

In [46]:
client.repository.list()

------------------------------------  ----------------  ------------------------  -----------------  -----
GUID                                  NAME              CREATED                   FRAMEWORK          TYPE
3eb9e4f2-a002-4367-b3c7-b41bb2d04fc8  LR xgboost Model  2021-01-10T13:19:18.002Z  scikit-learn_0.23  model
d005dc34-25d8-472e-8b2e-ba2dc47ad5b9  LR xgboost Model  2021-01-10T13:17:42.002Z  scikit-learn_0.23  model
69a4f290-3e67-49d0-99ee-62b66eb84ca6  LR xgboost Model  2021-01-10T13:17:13.002Z  scikit-learn_0.23  model
------------------------------------  ----------------  ------------------------  -----------------  -----


# Create Batch Deployment

In [47]:
client.deployments.ConfigurationMetaNames.show()

------------------------------  ----  --------  ----------
META_PROP NAME                  TYPE  REQUIRED  SCHEMA
TAGS                            list  N         ['string']
NAME                            str   N
DESCRIPTION                     str   N
CUSTOM                          dict  N
ASSET                           dict  N
HARDWARE_SPEC                   dict  N
HYBRID_PIPELINE_HARDWARE_SPECS  list  N
ONLINE                          dict  N
BATCH                           dict  N
R_SHINY                         dict  N
VIRTUAL                         dict  N
------------------------------  ----  --------  ----------


In [50]:
client.hardware_specifications.list()

-------------  ------------------------------------  --------------------------------------------------------------------------------------------------------------------------------------
NAME           ID                                    DESCRIPTION
ML             5a1f0e64-e420-55ce-bd7a-f6d00bb942cf  A hardware specification providing 4 CPU cores and 32 GiB of memory.
V100x2         a02f3ab5-6964-4f06-a870-c7cc69187895  A hardware specification providing 52 CPU cores and 96 GiB of memory with 2 Nvidia v100 GPUs.
L              a6c4923b-b8e4-444c-9f43-8a7ec3020110  A hardware specification providing 8 CPU cores and 32 GiB of memory.
Default Spark  ac59d20a-9c7c-4504-a853-788ef35969da  A hardware specification for Spark with 1 CPU and 4 GiB of memory for master and worker nodes, with 2 workers.
XXS            b128f957-581d-46d0-95b6-8af5cd5be580  A hardware specification providing one CPU core and 2 GiB of memory.
M-Spark        b2232f7a-bfad-4822-9bce-6ba1af49217a  A hardware specif

In [10]:
#create batch deployment
hardware_spec_id = client.hardware_specifications.get_id_by_name('XS')
deployment_meta = {
    client.deployments.ConfigurationMetaNames.NAME: "LR xboost deployment",
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 
        "id": hardware_spec_id,
        "nodes": 1
    },
    client.deployments.ConfigurationMetaNames.BATCH: {} 
}
model_uid = "3eb9e4f2-a002-4367-b3c7-b41bb2d04fc8"
deployment = client.deployments.create(model_uid, meta_props = deployment_meta)



#######################################################################################

Synchronous deployment creation for uid: '3eb9e4f2-a002-4367-b3c7-b41bb2d04fc8' started

#######################################################################################


ready.


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='d9cce8b3-e655-44d9-a7d0-017f5a2f8a5c'
------------------------------------------------------------------------------------------------




In [11]:
#get deployment uid
deployment_id = client.deployments.get_uid((deployment))

In [12]:
#get deployment detail
client.deployments.get_details(deployment_id)

{'entity': {'asset': {'id': '3eb9e4f2-a002-4367-b3c7-b41bb2d04fc8'},
  'batch': {},
  'custom': {},
  'deployed_asset_type': 'model',
  'hardware_spec': {'id': 'f3ebac7d-0a75-410c-8b48-a931428cc4c5',
   'name': 'XS',
   'num_nodes': 1},
  'name': 'LR xboost deployment',
  'space_id': '35e5769e-3a6a-4eba-8fa0-09ebd594f205',
  'status': {'state': 'ready'}},
 'metadata': {'created_at': '2021-01-17T12:31:34.688Z',
  'id': 'd9cce8b3-e655-44d9-a7d0-017f5a2f8a5c',
  'modified_at': '2021-01-17T12:31:34.688Z',
  'name': 'LR xboost deployment',
  'owner': 'IBMid-550009U7HH',
  'space_id': '35e5769e-3a6a-4eba-8fa0-09ebd594f205'}}

client.deployments.list()

# Job Deployment

In [ ]:
#feed input data to model deployment and get model output

In [9]:
client.data_assets.list()

------------  ----------  ----  ------------------------------------
NAME          ASSET_TYPE  SIZE  ASSET_ID
model-input   data_asset  0     4b33dc50-1db7-4609-bded-6294484429fb
model-output  data_asset  0     fc3454ba-1e8c-42c1-b8ba-9a2b5215495b
data input    data_asset  0     00edae82-9230-4516-a18a-b7aa10400268
------------  ----------  ----  ------------------------------------


In [24]:
model_input = client.data_assets.get_details('4b33dc50-1db7-4609-bded-6294484429fb')
model_input_id = client.data_assets.get_id(model_input)
model_input_href = client.data_assets.get_href(model_input)
model_input_id, model_input_href

('4b33dc50-1db7-4609-bded-6294484429fb',
 '/v2/assets/4b33dc50-1db7-4609-bded-6294484429fb?space_id=35e5769e-3a6a-4eba-8fa0-09ebd594f205')

In [25]:
model_output = client.data_assets.get_details('fc3454ba-1e8c-42c1-b8ba-9a2b5215495b')
model_output_id = client.data_assets.get_id(model_output)
model_output_href = client.data_assets.get_href(model_output)
model_output_id, model_output_href

('fc3454ba-1e8c-42c1-b8ba-9a2b5215495b',
 '/v2/assets/fc3454ba-1e8c-42c1-b8ba-9a2b5215495b?space_id=35e5769e-3a6a-4eba-8fa0-09ebd594f205')

In [47]:
client.deployments.ScoringMetaNames.show()

---------------------  ----  --------  --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
META_PROP NAME         TYPE  REQUIRED  SCHEMA
NAME                   str   N
INPUT_DATA             list  N         [{'name(optional)': 'string', 'id(optional)': 'string', 'fields(optional)': 'array[string]', 'values': 'array[array[string]]'}]
INPUT_DATA_REFERENCES  list  N         [{'id(optional)': 'string', 'name(optional)': 'string', 'type(required)': 'string', 'connection(required)': {'endpoint_url(required)': 'string', 'access_key_id(required)': 'string', 'secret_access_key(required)': 'string'}, 'locati

In [74]:
deploymentjob_meta = {
    client.deployments.ScoringMetaNames.NAME: "Test Predict use LR xgboost",
    client.deployments.ScoringMetaNames.INPUT_DATA_REFERENCES:[{ 
        'name': 'model-input-data', 
        'type': 's3', 
        'connection': {
            'endpoint_url': 'https://s3.us-south.cloud-object-storage.appdomain.cloud',
            'access_key_id': 'd9089fbf61194020b3601d7a24c5663e',
            'secret_access_key': '04d89ef816e54bff698cc2c85b1caa61434314fe5a8eb4a5'
        }, 
        'location': {
            'bucket': 'model-input-data', 
            #'path': '/data for predict.csv'
        }, 
    }],
    client.deployments.ScoringMetaNames.OUTPUT_DATA_REFERENCE:{ 
        'name': 'model-output-data', 
        'type': 's3', 
        'connection': {
            'endpoint_url': 'https://s3.us-south.cloud-object-storage.appdomain.cloud',
            'access_key_id': 'd9089fbf61194020b3601d7a24c5663e',
            'secret_access_key': '04d89ef816e54bff698cc2c85b1caa61434314fe5a8eb4a5'
        }, 
        'location': {
            'bucket': 'model-output-data', 
            #'path': '/result.csv'
        }, 
    }
}

#create job on that deployment_id
job = client.deployments.create_job(deployment_id, deploymentjob_meta)

In [39]:
deploymentjob_meta = {
 client.deployments.ScoringMetaNames.NAME: "Test Predict use LR xgboost",
 client.deployments.ScoringMetaNames.INPUT_DATA_REFERENCES: [{
     "id": model_input_id,
     "name": "model-input",
     "type": "data_asset",
     "connection": {},
     "location": {
            "href": model_input_id
     }
 }],
 client.deployments.ScoringMetaNames.OUTPUT_DATA_REFERENCE: {
         "id": model_input_id,
         "name": "model-output",
         "type": "data_asset",
         "connection": {},
         "location": {
             "href": model_output_id
         }
     }     
 }

deployment_id = 'cda1f8b8-45f6-40af-a89a-8df86b0f2584'

#create job on that deployment_id
job = client.deployments.create_job(deployment_id, deploymentjob_meta)

In [75]:
job_id = client.deployments.get_job_uid(job)
job_id

'82066f96-023b-4760-acbf-a52eb3dde8ec'

In [76]:
 client.deployments.get_job_status(job_id)

{'completed_at': '2021-01-11T04:23:18.534Z',
 'failure': {'trace': '57bdc055cc32148a939df785a3103bea',
  'errors': [{'code': 'python_runtime_out_of_memory',
    'message': "('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))"}]},
 'running_at': '',
 'state': 'failed'}

In [77]:
client.deployments.get_job_details(job_id)

{'entity': {'deployment': {'id': 'c138c0c8-6683-49a3-94d9-f988c16133c5'},
  'platform_job': {'job_id': 'e0b4d31e-f1d4-4f41-967c-0383f1fb22db',
   'run_id': '94975d8c-3a6f-484b-b8a3-fd94f111820f'},
  'scoring': {'input_data_references': [{'connection': {'access_key_id': 'd9089fbf61194020b3601d7a24c5663e',
      'endpoint_url': 'https://s3.us-south.cloud-object-storage.appdomain.cloud',
      'secret_access_key': '04d89ef816e54bff698cc2c85b1caa61434314fe5a8eb4a5'},
     'location': {'bucket': 'model-input-data'},
     'type': 's3'}],
   'output_data_reference': {'connection': {'access_key_id': 'd9089fbf61194020b3601d7a24c5663e',
     'endpoint_url': 'https://s3.us-south.cloud-object-storage.appdomain.cloud',
     'secret_access_key': '04d89ef816e54bff698cc2c85b1caa61434314fe5a8eb4a5'},
    'location': {'bucket': 'model-output-data'},
    'type': 's3'},
   'status': {'completed_at': '2021-01-11T04:23:18.534Z',
    'failure': {'trace': '57bdc055cc32148a939df785a3103bea',
     'errors': [{'